<a href="https://colab.research.google.com/github/mn22abk/Research-Methods-Assign.3/blob/main/Training_mid_size_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cell 1: Installing simpletransformers

In this cell, the `simpletransformers` package is installed using the following command:

In [ ]:
!pip install simpletransformers

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/42.4 kB ? eta -:--:--
     ------------------- -------------------- 20.5/42.4 kB ? eta -:--:--
     --------------------------- ---------- 30.7/42.4 kB 262.6 kB/s eta 0:00:01
     --------------------------- ---------- 30.7/42.4 kB 262.6 kB/s eta 0:00:01
     -------------------------------------- 42.4/42.4 kB 172.0 kB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------  41.0/42.0 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------  41.0/42.0 kB 2.0 MB/s eta 0:00:01
     ---------------------------------------  41.0/42.0 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 169.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/134.8 kB ? eta -:--:--
     --------------------------- ------------ 92.2/134.8 kB ? eta -:--:-

### Cell 2: Loading Data from JSON File

In this cell, data is loaded from a JSON file named "train.json" using the `json.load()` method.

In [ ]:
import json
with open(r"train.json", "r") as read_file:
    train = json.load(read_file)

In [ ]:
train

### Cell 4: Loading Data from JSON File

In this cell, data is loaded from a JSON file named "test.json" using the `json.load()` method.

In [ ]:
with open(r"test.json", "r") as read_file:
    test = json.load(read_file)

In [ ]:
test

### Cell 6: Setting up Question Answering Model

In this cell, the necessary libraries are imported and user warnings of the `UserWarning` category are ignored using the `warnings.filterwarnings("ignore", category=UserWarning)` method.

In [ ]:
import logging
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

### Cell 7: Selecting Pretrained Model

In this cell, a pretrained model is selected based on the specified `model_type`.

In [ ]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

### Cell 8: Configuring the Model

In this cell, the model is configured with specific settings using `QuestionAnsweringArgs()`.

In [ ]:
# Configure the model
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=8

### Cell 9: Advanced Training Methodology

In this cell, advanced training methodology is defined using a dictionary named `train_args`.

In [ ]:
### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 8,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    "train_batch_size": 128,
    "eval_batch_size": 64,
}

### Cell 10: Initializing Question Answering Model

In this cell, a Question Answering Model is initialized using the specified `model_type`, `model_name`, and training arguments (`train_args`).

In [ ]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args, use_cuda=False
)

In [ ]:
### Remove output folder
!rm -rf outputs

### Cell 12: Training the Model

In this cell, the initialized model is trained using the `train` data and evaluated on the `test` data.

In [ ]:
# Train the model
model.train_model(train, eval_data=test)

### Cell 13: Evaluating the Model

In this cell, the trained model is evaluated on the `test` data.

In [ ]:
# Evaluate the model
result, texts = model.eval_model(test)

### Cell 14: Making Predictions with the Model

In this cell, predictions are made using the trained model on the given context and questions.

In [ ]:
# Make predictions with the model
to_predict = [
    {
        "context": "Vin is a Mistborn of great power and skill.",
        "qas": [
            {
                "question": "What is Vin's speciality?",
                "id": "0",
            }
        ],
    }
]

### Cell 15: Generating Predictions

In this cell, predictions are generated using the trained model on the provided input.

In [ ]:
answers, probabilities = model.predict(to_predict)

print(answers)